In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
#import pycaret.classification as pc
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [2]:
df = dd.read_parquet('..\\full_data\\figma_plan\\full_gender_data.parquet').compute()

# добавляем target, удаляем nan
df = df.merge(pd.read_parquet('..\\full_data\\public_train.pqt',columns=['user_id','is_male'])[['is_male','user_id']], on = 'user_id', how = 'inner')
df = df.loc[~(df['is_male'].isna()) & (df['is_male'] != 'NA')]
df['is_male'] = df['is_male'].astype('int8')

display(df.head())
display(df.info())

,user_id,region_name_0,region_name_1,region_name_2,region_name_3,region_name_4,region_name_5,region_name_6,region_count,city_name_0,...,bukvaprava.ru,belnovosti-by.turbopages.org,ngs24-ru.turbopages.org,forum.mfd.ru,yomed.ru,7info.ru,other_url,other_female_urls_frac,other_male_urls_frac,is_male
0,0,0,0,0,0,0,0,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.031088,0.005181,0.000000,0
1,1,0,0,0,0,0,0,1,3,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.013372,0.001910,0.000955,0
2,2,0,0,0,0,0,1,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.002433,0.000000,0.000000,0
3,3,0,0,0,0,0,1,1,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.010909,0.003636,0.003636,1
4,4,0,0,0,0,1,0,0,5,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.011583,0.003861,0.000000,0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 264326 entries, 0 to 269999
Columns: 3038 entries, user_id to is_male
dtypes: category(26), float32(3008), int32(1), int8(3)
memory usage: 3.0 GB


None

In [ ]:
df = df.drop('price',axis=1)
'''nan_urls_frac = df.isna().sum()[df.isna().sum() > 0]
display(nan_urls_frac.head())

display(df.loc[df.price.isna(),['cpe_manufacturer_name_0',
                                'cpe_manufacturer_name_1',
                                'cpe_manufacturer_name_2',
                                'cpe_manufacturer_name_3',
                                'cpe_manufacturer_name_4',
                                'cpe_manufacturer_name_5']])

df.loc[(df['cpe_manufacturer_name_0'] == 0) &
       (df['cpe_manufacturer_name_1'] == 0) &
       (df['cpe_manufacturer_name_2'] == 0) &
       (df['cpe_manufacturer_name_3'] == 0) &
       (df['cpe_manufacturer_name_4'] == 1) &
       (df['cpe_manufacturer_name_5'] == 0), 'price'].dropna().median()'''

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df.drop(['is_male','user_id'], axis = 1),df['is_male'], test_size = 0.33, random_state = 42)

Pycaret

In [ ]:
#df = df.drop('user_id',axis=1)
s = pc.setup(df, target = 'is_male', session_id = 123,fold=2)

In [ ]:
best = pc.compare_models(include = ['lightgbm', 'catboost'],fold=2)

Дефолт градиент

In [ ]:
model = GradientBoostingClassifier(random_state=42).fit(x_train,y_train)

In [15]:
print(f'GINI по полу {2 * roc_auc_score(y_test, model.predict_proba(x_test)[:,1]) - 1:2.3f}')

GINI по полу 0.682


In [13]:
print(classification_report(y_test, model.predict(x_test)))

              precision    recall  f1-score   support

           0       0.75      0.75      0.75     42609
           1       0.76      0.76      0.76     44619

    accuracy                           0.76     87228
   macro avg       0.76      0.76      0.76     87228
weighted avg       0.76      0.76      0.76     87228



GridSearchCV

In [ ]:
parameters = {'C': np.linspace(0.5, 1.5, 11)}
clf = GridSearchCV(GradientBoostingClassifier(random_state=42), parameters, cv=5, verbose=5)# n_jobs=-1
clf.fit(df.drop('is_male',axis=1), df['is_male'])